In [3]:
from datetime import date
import sys
sys.path.append(r".")
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader

from time_series_model.train import train, test
from time_series_model.args import Model_args
from data.get_db import get_data_period
from statsmodels.tsa.seasonal import seasonal_decompose
from time_series_model.utils import *

In [9]:
start_date = date(2023, 6, 14)
end_date = date(2023, 6, 20)

In [11]:
preprocess_seq(args, model_name, start_date, end_date)

NameError: name 'preprocess_seq' is not defined

In [7]:
start_date = date(2023, 6, 14)
end_date = date(2023, 6, 20)
indoor_data = get_data_period('indoor', start_date, end_date)
indoor_data = pd.DataFrame(indoor_data, columns=["Date", "temp", "humi", "CO2"])
indoor_data

,Date,temp,humi,CO2
0,2023-06-14 00:00:00,27.4,70.0,303.0
1,2023-06-14 00:01:00,27.4,70.0,302.0
2,2023-06-14 00:02:00,27.4,70.0,301.0
3,2023-06-14 00:03:00,27.4,70.0,302.0
4,2023-06-14 00:04:00,27.4,70.1,301.0
...,...,...,...,...
9895,2023-06-20 23:55:00,28.3,68.2,413.0
9896,2023-06-20 23:56:00,28.3,68.2,413.0
9897,2023-06-20 23:57:00,28.3,68.2,409.0
9898,2023-06-20 23:58:00,28.3,68.2,412.0


In [8]:
"""
Obtain data from the database and perform data preprocessing.

Args:
    seq_len: seq len.
    model_name: model name.
    start_date:  data start date.
    end_date: data end date.

Returns:
    The dataframe required by the corresponding model.
"""

# 溫度及相關資料

#室內資料
indoor_data = get_data_period('indoor', start_date, end_date)
indoor_data = pd.DataFrame(indoor_data, columns=["Date", "temp", "humi", "CO2"])
# 將要 predict 的 column 移動到第一行
indoor_data = adjust_col(indoor_data, "temp")
# 篩選時間區段
indoor_data = filter_time(indoor_data)
print(indoor_data)

#控制資料
controller_data = get_data_period('controller', start_date, end_date)
controller_data = pd.DataFrame(controller_data, columns=["Date", "I_O", "ac_temp", "fan", "mode"])
# 篩選時間區段
controller_data = filter_time(controller_data)
# 沒有開啟冷氣機時的數值更改
controller_data.loc[controller_data['I_O'] == 0, 'ac_temp'] = 0
controller_data.loc[controller_data['I_O'] == 0, 'fan'] = 3
controller_data.loc[controller_data['I_O'] == 0, 'mode'] = 3
print(controller_data)

#室外資料
outdoor_data = get_data_period('outdoor', start_date, end_date)
outdoor_data = pd.DataFrame(
    outdoor_data,
    columns=["Date",
                "WDIR",
                "WDSD",
                "TEMP",
                "HUMD",
                "PRES",
                "RAIN",
                "HFX",
                "HXD",
                "HF",
                "HD",
                "HUVI",
                "WEATHER"]
)
# 篩選時間區段
outdoor_data = filter_time(outdoor_data)
print(outdoor_data)

#功耗資料
#冰水機資料
chiller_data = get_data_period('CS_nor', start_date, end_date)
chiller_data = pd.DataFrame(
    chiller_data,
    columns=[
        "date",
        "time",
        "tM-TH8_AI.sensor0",
        "tM-TH8_AI.sensor1",
        "tM-TH8_AI.sensor2",
        "tM-TH8_AI.sensor3",
        "tM-TH8_AI.sensor4",
        "tM-TH8_AI.sensor5",
        "tM-TH8_AI.sensor6",
        "tM-TH8_AI.sensor7",
        "PM-3133_AI.V",
        "PM-3133_AI.I",
        "PM-3133_AI.PF",
        "PM-3133_AI.Kwh",
        "PM-3133_AI.Kvarh",
        "PM-3133_AI.Kvah"
    ]
)
chiller_data['time'] = pd.to_timedelta(chiller_data['time'])
# 提取時間部分並以字串格式保存
chiller_data['time'] = chiller_data['time'].dt.total_seconds(
).apply(lambda x: pd.to_datetime(x,
                                    unit='s').strftime('%H:%M:%S'))
# 合併 date 和 time 列的數值
chiller_data['Date'] = (chiller_data['date'].astype(str) + ' ' + chiller_data['time'].astype(str))
# 刪除原來的 date 和 time 列
chiller_data = chiller_data.drop(['date', 'time'], axis=1)
# 將要 predict 的 column 移動到第一行
chiller_data = adjust_col(chiller_data, "PM-3133_AI.Kwh")
# 篩選時間區段
chiller_data = filter_time(chiller_data)
# 蒸發器差值
chiller_data = make_features(chiller_data, 'tM-TH8_AI.sensor0', 'tM-TH8_AI.sensor5')
# 壓縮機差值
chiller_data = make_features(chiller_data, 'tM-TH8_AI.sensor1', 'tM-TH8_AI.sensor2')
# 膨脹閥差值
chiller_data = make_features(chiller_data, 'tM-TH8_AI.sensor6', 'tM-TH8_AI.sensor7')
# 調整有功功率數值
chiller_data = make_features(chiller_data, 'PM-3133_AI.Kwh', None)
# 調整無功功率數值
chiller_data = make_features(chiller_data, 'PM-3133_AI.Kvarh', None)
# 調整視在功率數值
chiller_data = make_features(chiller_data, 'PM-3133_AI.Kvah', None)
print(chiller_data)

dataset = indoor_data.merge(outdoor_data, on='Date', how='outer')
dataset = dataset.merge(controller_data, on='Date', how='outer')
dataset = dataset.merge(chiller_data, on='Date', how='outer')
#dataset = seasonal_decomp(seq_len, dataset, 'temp')

dataset = dataset.fillna(method='ffill')
# 第一天沒有開啟冷氣機時的數值更改
dataset.loc[np.isnan(dataset['I_O']), 'ac_temp'] = 0
dataset.loc[np.isnan(dataset['I_O']), 'fan'] = 3
dataset.loc[np.isnan(dataset['I_O']), 'mode'] = 3
dataset.loc[np.isnan(dataset['I_O']), 'I_O'] = 0
print(dataset.isnull().any().any())
print(dataset)



NameError: name 'adjust_col' is not defined